In [28]:
import pandas as pd
df=pd.read_csv('blogs_categories.csv')
df

,Unnamed: 0,Data,Labels
0,0,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:49...,alt.atheism
1,1,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:51...,alt.atheism
2,2,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,alt.atheism
3,3,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:51...,alt.atheism
4,4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:51...,alt.atheism
...,...,...,...
19992,19992,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:54...,talk.religion.misc
19993,19993,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:54...,talk.religion.misc
19994,19994,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc
19995,19995,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc


In [3]:
df.isnull().sum()

Unnamed: 0    0
Data          0
Labels        0
dtype: int64

In [5]:
df[df.duplicated()]

,Unnamed: 0,Data,Labels


In [6]:
df.dtypes

Unnamed: 0     int64
Data          object
Labels        object
dtype: object

In [29]:
# prompt: take Data column and convert every row into lower case and remove puncatations

# Convert text to lowercase and remove punctuation
df['Data'] = df['Data'].str.lower()

df['Cleaned_Text'] = df['Data'].str.replace('[^\w\s]', '', regex=True)
df['Cleaned_Text']

0        xref cantaloupesrvcscmuedu altatheism49960 alt...
1        xref cantaloupesrvcscmuedu altatheism51060 alt...
2        newsgroups altatheism\npath cantaloupesrvcscmu...
3        xref cantaloupesrvcscmuedu altatheism51120 alt...
4        xref cantaloupesrvcscmuedu altatheism51121 soc...
                               ...                        
19992    xref cantaloupesrvcscmuedu altatheism54482 tal...
19993    xref cantaloupesrvcscmuedu altatheism54485 tal...
19994    xref cantaloupesrvcscmuedu talkreligionmisc845...
19995    xref cantaloupesrvcscmuedu talkreligionmisc845...
19996    xref cantaloupesrvcscmuedu talkabortion121820 ...
Name: Cleaned_Text, Length: 19997, dtype: object

In [30]:
from nltk.tokenize import word_tokenize

df['Tokenized_Text'] = df['Cleaned_Text'].apply(word_tokenize)
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
df['Filtered_Text'] = df['Tokenized_Text'].apply(lambda x: [word for word in x if word not in stop_words])

# import re
# # removing numbers 
# df['Filtered_Text'] = df['Filtered_Text'].apply(lambda x: ' '.join([str(elem) for elem in x if not re.match(r'\d+', str(elem))]))

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(df['Filtered_Text'].apply(lambda x: ' '.join(x)))

In [36]:
print(X_tfidf)

AttributeError: 'csr_matrix' object has no attribute 'len'

In [26]:
# Convert the TF-IDF matrix to a DataFrame for easier analysis
tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

# Print the TF-IDF DataFrame
print(tfidf_df)

MemoryError: Unable to allocate 38.8 GiB for an array with shape (19997, 260567) and data type float64

In [ ]:
from sklearn.model_selection import train_test_split

X = df['Cleaned_Text']  # Features
y = df['Labels']         # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Define a pipeline for text classification
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB()),
])

# Train the classifier
text_clf.fit(X_train, y_train)

# Make predictions
y_pred = text_clf.predict(X_test)
from sklearn.metrics import classification_report, accuracy_score

# Print classification report
print(classification_report(y_test, y_pred))

# Print accuracy score

In [15]:
import re

df['Data'] = df['Data'].apply(lambda x: ' '.join([str(elem) for elem in x if not re.match(r'\d+', str(elem))]))


In [16]:

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Define stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
  tokens = word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
  return ' '.join(tokens)

df['Data'] = df['Data'].apply(preprocess_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jalli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jalli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
# prompt: apply TF-IDF

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate the TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the preprocessed text data
tfidf_matrix = vectorizer.fit_transform(df['Data'])

# Convert the TF-IDF matrix to a DataFrame for easier analysis
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Print the TF-IDF DataFrame
print(tfidf_df)


MemoryError: Unable to allocate 29.2 GiB for an array with shape (19997, 195814) and data type float64

In [8]:
# import pandas as pd
# import string
# from sklearn.feature_extraction.text import TfidfVectorizer
# import nltk
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer

# # Define stopwords and lemmatizer
# stop_words = set(stopwords.words('english'))
# lemmatizer = WordNetLemmatizer()

# # Ensure nltk data is downloaded
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# # Define a set of common stopwords
# common_stopwords = set([
#     'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 
#     'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 
#     'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 
#     'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 
#     'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 
#     'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 
#     'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 
#     'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 
#     'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now'
# ])

# # Function to preprocess text
# def preprocess_text(text):
#     # Convert text to lower case
#     text = text.lower()
#     # Remove punctuation
#     text = text.translate(str.maketrans('', '', string.punctuation))
#     # Tokenize text
#     tokens = text.split()
#     # Remove stopwords and apply lemmatization
#     tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
#     return ' '.join(tokens)

# # Apply preprocessing to the 'Data' column
# df['Data'] = df['Data'].apply(preprocess_text)
# # print(df['Data'])
# # Sample a smaller subset of the data to avoid memory issues
# sampled_df = df.sample(n=1000, random_state=42)

# # Apply TF-IDF feature extraction on the sampled data
# vectorizer = TfidfVectorizer()
# tfidf_matrix_sampled = vectorizer.fit_transform(sampled_df['Data'])

# # Convert the TF-IDF matrix to a DataFrame for easier inspection
# tfidf_df_sampled = pd.DataFrame(tfidf_matrix_sampled.toarray(), columns=vectorizer.get_feature_names_out())

# # Display the first few rows of the TF-IDF DataFrame
# print(tfidf_df_sampled)


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_df, df['Labels'], test_size=0.2)

# Create a Multinomial Naive Bayes classifier
clf = MultinomialNB()

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
predicted_labels = clf.predict(X_test)

# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test, predicted_labels)

# Print the accuracy
print("Accuracy:", accuracy)


MemoryError: Unable to allocate 5.83 GiB for an array with shape (4000, 195751) and data type float64